# pyPCGA stwave inversion example

In [ ]:
%matplotlib inline

- import relevant python packages after installing pyPCGA
- stwave.py includes python wrapper to stwave model

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib
from scipy.io import savemat, loadmat
import numpy as np
import stwave as st
from pyPCGA import PCGA
import math
import datetime as dt
import os
import stwave_plot_utils as stplot

In [ ]:
matplotlib.rcParams['font.size'] = 16

In [ ]:
basedir = os.getcwd()
workdir=basedir#os.path.join(basedir,'results','duck','run01')

In [ ]:
os.chdir(workdir)

In [ ]:
uas_dir =  os.path.join('uas','LENKF_uas_c_and_topo_20160722_2030_f_v20180130')


- model domain and discretization

In [ ]:
Lx = 500.0
Ly = 300.0
nx = 100
ny = 60
x0, y0 = (80., 80.)
z0 = 0.
dx0 = (Lx-x0)/float(nx)

N = np.array([nx,ny])
m = np.prod(N) 
dx = np.array([dx0,dx0])
xmin = np.array([0. + dx[0]/2., 0. + dx[1]/2.])
xmax = np.array([x0+Lx*dx[0] - dx[0]/2., y0+Ly*dx[1] - dx[1]/2.])

- covariance kernel and scale parameters following Hojat's CSKF paper

In [ ]:
prior_std = 1.5
prior_cov_scale = np.array([100, 100])
def kernel(r): return (prior_std**2)*np.exp(-r**2)

- grid coordinates for plotting purposes

In [ ]:
x = np.linspace(x0 + dx[0]/2., x0+nx*dx[0] - dx[0]/2., N[0])
y = np.linspace(y0 + dx[1]/2., y0+ny*dx[1] - dx[0]/2., N[1])
XX, YY = np.meshgrid(x, y)
pts = np.hstack((XX.ravel()[:,np.newaxis], YY.ravel()[:,np.newaxis]))

- load data, true field is optional

In [ ]:
wave_obs_file  = os.path.join(uas_dir,'observation_files','wave_speed_measurements.txt')
topo_obs_file  = os.path.join(uas_dir,'observation_files','topo_measurements.txt')


In [ ]:
#observation indices
wave_obs_inds_file  = os.path.join(uas_dir,'observation_files','wave_speed_measurement_cells.txt')
topo_obs_inds_file  = os.path.join(uas_dir,'observation_files','topo_measurement_cells.txt')
obs_inds  = np.loadtxt(obs_inds_file,dtype='i')


In [ ]:
include_topo_obs = True

obs = np.loadtxt(wave_obs_file)
wave_obs_inds  = np.loadtxt(wave_obs_inds_file,dtype='i')
topo_obs_inds = None
if include_topo_obs:
    topo_obs = np.loadtxt(topo_obs_file)
    topo_obs = z0-topo_obs #convert to bathymetry
    obs = np.append(obs,topo_obs)
    topo_obs_inds = np.loadtxt(topo_obs_inds_file,dtype='i')


In [ ]:
elev_file = os.path.join(uas_dir,'observation_files','true_bathymetry.txt')
s_true = np.loadtxt(elev_file)
s_true = z0-s_true
#observations extracted onto stwave grid

- define domain extent, discretization and measurement collection time

In [ ]:
t1 = dt.datetime(2016, 07, 22, 19, 30)
t2 = dt.datetime(2016, 07, 22, 20, 30)

stwave_params = {'nx': nx, 'ny': ny, 'Lx': Lx, 'Ly': Ly, 'x0': x0, 'y0': y0, 't1': t1, 't2': t2,
          'offline_dataloc':"./input_files/FRF-ocean_waves_8m-array_201607.nc"}

- parameters for the simulation

In [ ]:
params = {'R':(0.2)**2, 'n_pc':200,
          'maxiter':10, 'restol':0.01,
          'matvec':'FFT','xmin':xmin, 'xmax':xmax, 'N':N,
          'prior_std':prior_std,'prior_cov_scale':prior_cov_scale,
          'kernel':kernel, 'post_cov':"diag",
          'precond':True, 'LM': True,
          'parallel':True, 'linesearch' : True,
          'forward_model_verbose': False, 'verbose': False,
          'iter_save': True}

- initial guess

In [ ]:
s_init = np.mean(s_true)*np.ones((m,1))

- initialize PCGA object

- run get inversion results

In [ ]:
ls -l shat*.txt

In [ ]:
s_hat     = np.loadtxt('shat3.txt')
simul_obs = np.loadtxt('simulobs3.txt')
post_diagv= np.loadtxt('postv.txt')

In [ ]:
# converting to 2d array for plotting
s_hat2d = s_hat.reshape(N[1],N[0])
s_true2d = s_true.reshape(N[1],N[0])
post_diagv[post_diagv <0.] = 0. # just in case
post_std = np.sqrt(post_diagv)
post_std2d = post_std.reshape(N[1],N[0])
error = s_true2d-s_hat2d

- plot results

In [ ]:
minv = s_true.min()
maxv = s_true.max()

fig, axes = plt.subplots(1,2, figsize=(15,5))
plt.suptitle('prior var.: (%g)^2, n_pc : %d' % (prior_std,params['n_pc']))
im = axes[0].imshow(np.flipud(np.fliplr(-s_true2d)), extent=[0, nx, 0, ny], vmin=-7., vmax=maxv, cmap=plt.get_cmap('jet'))
axes[0].set_title('(a) True', loc='left')
axes[0].set_aspect('equal')
axes[0].set_xlabel('Offshore distance (px)')
axes[0].set_ylabel('Alongshore distance (px)')
axes[1].imshow(np.flipud(np.fliplr(-s_hat2d)), extent=[0, nx, 0, ny], vmin=-7., vmax=maxv, cmap=plt.get_cmap('jet'))
axes[1].set_title('(b) Estimate', loc='left')
axes[1].set_xlabel('Offshore distance (px)')
axes[1].set_aspect('equal')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)


- plot uncertainty

In [ ]:
fig = plt.figure()
im = plt.imshow(np.flipud(np.fliplr(post_std2d)), extent=[0, nx, 0, ny], cmap=plt.get_cmap('viridis'))
plt.title('Uncertainty (std)', loc='left')
plt.xlabel('Offshore distance (px)')
plt.ylabel('Alongshore distance (px)')
plt.gca().set_aspect('equal', adjustable='box')
fig.colorbar(im)



- plot versus physical distance

In [ ]:
fig, axes = stplot.plot_2x1(nx,ny,dx[0],dx[1],s_true2d,s_hat2d,cmap_type='viridis')



In [ ]:
fig, axes = stplot.plot_2x1(nx,ny,dx[0],dx[1],s_hat2d,error,tit0='Estimate',tit1='Error',colorbar_flag=1,
                            vmin=[-7.,error.min().min()],vmax=[0.,error.max().max()],cmap_type='viridis')



In [ ]:
fig, axes = stplot.plot_1x1(nx,ny,dx[0],dx[1],error,tit0='Error',
                            vmin=[error.min().min()],vmax=[error.max().max()],cmap_type='coolwarm')



- plot transect at y = 25 px and 45 px

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1,adjustable='box',aspect=20)
ax = stplot.plot_transect(nx,ny,dx[0],dx[1],s_hat2d,s_true2d,post_std2d,25,axes=ax)


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1,adjustable='box',aspect=20)
ax = stplot.plot_transect(nx,ny,dx[0],dx[1],s_hat2d,s_true2d,post_std2d,5,axes=ax)


In [ ]:
fig = plt.figure(figsize=(10,10))

fig.suptitle('transect with prior var.: (%g)^2, n_pc : %d, lx = %f m, ly = %f m\n\n' % (prior_std, params['n_pc'],prior_cov_scale[0],prior_cov_scale[1]))

linex,line1_true,line1,line1_u,line1_l = stplot.get_transect(nx,ny,dx[0],dx[1],s_hat2d,s_true2d,post_std2d,25)
linex,line2_true,line2,line2_u,line2_l = stplot.get_transect(nx,ny,dx[0],dx[1],s_hat2d,s_true2d,post_std2d,45)
axes = []
axes.append(fig.add_subplot(2,1,1,adjustable='box',aspect=20))
axes[0] = stplot.plot_transect(nx,ny,dx[0],dx[1],s_hat2d,s_true2d,post_std2d,45,axes=axes[0])

axes.append(fig.add_subplot(2,1,2,adjustable='box',aspect=20))
axes[1] = stplot.plot_transect(nx,ny,dx[0],dx[1],s_hat2d,s_true2d,post_std2d,70,axes=axes[1])

#plt.savefig('uas_20160722_std1p5_npc200_ix45ix70.png')

In [ ]:
nobs = obs.shape[0]
fig = plt.figure()
plt.title('obs. vs simul.')
plt.plot(obs,simul_obs,'.',markersize=4)
plt.xlabel('observation')
plt.ylabel('simulation')
minobs = np.vstack((obs,simul_obs)).min().min()
maxobs = np.vstack((obs,simul_obs)).max().max()
plt.plot(np.linspace(minobs,maxobs,20),np.linspace(minobs,maxobs,20),'k-',linewidth=3)
plt.axis('equal')
axes = plt.gca()
axes.set_xlim([math.floor(minobs),math.ceil(maxobs)])
axes.set_ylim([math.floor(minobs),math.ceil(maxobs)])

In [ ]:
wave_obs_locs_file= os.path.join(uas_dir,'observation_files','wave_speed_measurement_locations.dat')
topo_obs_locs_file= os.path.join(uas_dir,'observation_files','topo_measurement_locations.dat')

In [ ]:
wave_obs_locs=np.loadtxt(wave_obs_locs_file)
obs_locs = wave_obs_locs
if include_topo_obs:
    topo_obs_locs=np.loadtxt(topo_obs_locs_file)
    obs_locs = np.append(obs_locs,topo_obs_locs,axis=0)

## Need to get coordinates right

In [ ]:
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(1,2,1,adjustable='box')
im = ax1.imshow(np.flipud(np.fliplr(-s_true2d)), extent=[x0+0.5*dx[0], x0+dx[0]*nx-dx[0]*0.5, y0+dx[1]*0.5, y0+ny*dx[1]-dx[1]*0.5], cmap=plt.get_cmap('jet'))
ax.set_title('Observation locations', loc='left')
ax.set_xlabel('Offshore distance (px)')
ax.set_ylabel('Alongshore distance (px)')
ax.set_aspect('equal', adjustable='box')

ax2 = fig.add_subplot(1,2,2,adjustable='box')
im2 = ax2.imshow(np.flipud(np.fliplr(post_std2d)), extent=[x0+0.5*dx[0], x0+dx[0]*nx-dx[0]*0.5, y0+dx[1]*0.5, y0+ny*dx[1]-dx[1]*0.5], cmap=plt.get_cmap('jet'))
ax2.scatter(obs_locs[:,0],obs_locs[:,1],alpha=0.5)

£plt.savefig('uas_20160722_std1p5_npc200_true_unc_with_topo_obs.png')

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(1,1,1,projection='3d')
ax1.scatter3D(obs_locs[:,0],obs_locs[:,1],obs)
ax1.view_init(90,-90)
